In [7]:
import pandas as pd
import numpy as np

X = pd.read_pickle(f"../input.pkl")
y = pd.read_pickle(f"../output.pkl")

In [8]:
import torch.cuda
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


True
NVIDIA GeForce RTX 4070 SUPER


In [10]:
import xgboost as xgb
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from tabpfn import TabPFNClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

scalar=StandardScaler()
X_train_pr=scalar.fit_transform(X_train)
X_test_pr=scalar.transform(X_test)

X_trainp = pd.DataFrame(X_train_pr, columns=X_train.columns, index=X_train.index)
X_testp = pd.DataFrame(X_test_pr, columns=X_test.columns, index=X_test.index)


    # 모델 정의
preprocess = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(),  ['age', 'educ', 'house', 'pov', 'wt', 'ht', 'bmi', 'wst', 'hip', 'dia', 'pulse', 'sys', 'alt', 'albumin', 'ast', 'crea', 'chol', 'tyg', 'ggt', 'wbc', 'hb', 'hct', 'ldl', 'hdl', 'acratio', 'glu', 'insulin', 'crp', 'mvpa', 'ac_week']),
            ("cat", OneHotEncoder(handle_unknown="ignore"), ['gender','marry','race'])
        ]
    )

    # ============================================
    # 3. TabPFN v2 모델 정의
    # ============================================
model = TabPFNClassifier(
        device="auto",  # GPU 사용 가능 시 "cuda"
    )

    # ============================================
    # 4. Pipeline 구성
    # ============================================
pipe = Pipeline([
        ("preprocess", preprocess),
        ("tabpfn", model)
    ])

    # ============================================
    # 5. 모델 학습
    # ============================================
pipe.fit(X_trainp, y_train)


# 테스트셋 예측 및 평가
y_pred = pipe.predict(X_testp)    #이거 하면 [0.3,0.6,0.1]와 같이 prob 결과를 산출한 후 가장 값이 큰 인덱스를 y_pred에 넣는다. 
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred,average='weighted')
rec=recall_score(y_test,y_pred,average='weighted')
f1=f1_score(y_test,y_pred,average='weighted')

print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1 score: {f1:.4f}")


c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9664, Precision: 0.9676, Recall: 0.9664, F1 score: 0.9618
